# Wrangle Report - WeRateDogs twitter data
#### Author: Devin McCormack

## Gathering
WeRateDogs twitter data was assembled from three sources:

1. Enhanced twitter archive from local file `twitter_archive_enhanced.csv`, imported as a dataframe name `df_dog`, with 2356 observations of 17 variables.
2. Udacity hosted Neural Network image predictions file `image-predictions.tsv` requested programmatically from [this link.](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv), imported as a dataframe named `df_breed` with 2075 observations of 12 variables.
3. additional tweet data (retweets, likes, etc.) requested from twitter API using Tweepy Library, compiled in local file `tweet_json.txt` with each tweet JSON data written to a newline. Imported as a dataframe named `df_tweet` with 2347 observations of 4 selected variables (`tweet_id`, `favorite_count`,`retweet_count`,and `followers_count`).

## Assessment
After gathering, the data was assessed for tidyness and quality across four dimensions: Completeness, Validity, Accuracy, and Consistency.


1. Completeness

    - `df_dog` contained 2356 unique tweet ids; of them, 78 were replies, and 181 were retweets. 
    - `df_breed` contained 2075 unique tweet ids, corresponding to tweets in `df_dog` posted before August 1, 2017. 
    - `df_tweet`, which was gathered February 2018 using `df_dog` tweet ids and the twitter API, returned only 2347 tweets. The discrepancy between `df_tweet` and `df_dog` is due to tweet deletion before API access. 
    - `df_dog` contained true null values in 6 variables: `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeeted_status_user_id`, `retweeted_status_timestamp`, and `expanded_urls`. 
    - `df_dog` also included null values encoded as string "None" in `name`, `doggo`, `floofer`, `pupper` and `puppo`.
    
2. Validity

    - `df_dog` variable `name` encodes null values as string "None".
    - `df_dog` variable `timestamp` is a object instead of datetime.
    - `df_dog` variables `doggo`, `floofer`, `pupper` and `puppo` are a messy form of a single variable named `dog_stage`.
    - `df_dog` variables `rating_numerator` and `rating_denominator` are a messy form of a single ratio `rating`.
    - All retweets and replies, marked with values in respective `..._status_id` variables, are invalid representations of original tweeted images/ratings.
    - primary key tweet_id is valid (no duplicates) across all datasets, but was extracted in `df_tweet` as a string to ensure integrity in the API request process.

3. Accuracy

    - `df_dog` contained string extracted data in the variables `name`, `rating_numerator`, and `rating_denominator`. In various cases, these values were incorrectly extracted.
    - `df_breed` indicates that some images are not classified as dog breeds. In some cases, this is an inaccurate classification due to challenging image composition, and in other cases the images were in fact not of dogs at all. This indicates inaccuracy in both classification and in image description. *However*, it is worth noting that WeRateDogs takes the stance that "anything can be a dog, some things are just worse dogs than others". 
    
4. Consistency

    - This dataset is consistent across the three tables.
    
    

## Cleaning

Clean copies of the data tables were created (`df_dog_clean`,`df_tweet_clean`, and `df_breed_clean` respectively), and data was determined to be as complete as possible (excluding incomplete datapoints due to inaccurate string extraction).

***

First tidying of "dog stages" that were spread across four variables. 

Some tweets had more than one "dog stage", an issue of quality. Due to the low number of images with multiple dog stages, any image with multiple stages can be classified within a single group "multiple".

In order to create the new dog_stages variable, I concatenated the four variable values into one string value. For example, a tweet that only included one keyword, "pupper" would contain the value "NoneNonepupperNone". Then, with a dictionary of string replacements, I replaced these strings in `df_stage` with the appropriate keyword ("pupper"in the examples given). After replacement of simple category strings, A regular expression was then used to replace all values with mutliple keywords with the keyword "multiple". After verifying new variable `dog_stage`, variables `doggo`, `floofer`, `pupper`, `puppo` were dropped from `df_dog_clean`.

***

Further tidying is done by inner joining the three tables on the variable `tweet_id`. This automatically reduces the dataset to only the tweet ids that has complete data across all three tables. 

Before doing so, I needed to convert `df_tweet_clean.tweet_id` to a 64bit integer to match the other tables.

After merging, I had a single table with 2070 observations of 28 variables, called `df_dog_clean`.

***

To remove retweets and replies, any tweet id that had non-null values in either `in_reply_to_status_id`, or `retweeted_status_id` were removed. After removing, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeeted_status_user_id`, and `retweeted_status_timestamp` were dropped from `df_dog_clean`, resulting in a table with 1971 observations of 23 variables.

***

Quality of `name` was cleaned next. After probing and iterating, I created a regex solution to extract names from 1380 of 1971 observations, with fewer false positives and negatives than the original extraction. Reducing false negatives to zero would require complex regex extraction, or extensive manual modification.

Additionally, this extraction delt with a quality issue by assigning null values to images without extracted names.

***

Next, `rating_numerator` and `rating_denominator` were re-extracted. One tweet was found to not have a rating at all, and was dropped from `df_dog_clean`. Accurate denominators were found to always be multiples of 10, and to often be the very last fraction present in the text. Numerators were extracted by proximity to the accurate denominators. 

After inspection, There were 24 tweets that potentially had mutliple valid ratings for multiple dogs within the image. The numerator and denominator for these 24 images was manually exracted and updated in `df_dog_clean`.

***

`rating_numerator` and `rating_denominator` were then made tidy by combining them into a single variable named `rating` by dividing `rating_numerator` by `rating_denominator`. After testing, `rating_numerator` and `rating_denominator`  were dropped from `df_dog_clean`, resulting in 22 variables.

***

images where `p1_dog`,`p2_dog`, and `p3_dog` were False were spot checked. Visually inspecting these images seem to show a big challenge with NN image classification: if the dog is not the largest object in the image or blends in, the NN has issues "focusing" on the dog to make a classification. Some of the images are purposefully not dogs.

This was determined to be something that *did not* need to be cleaned, however it should be kept in mind when aggregating data to make insights. 

***

Finally, the datatype of `timestamp` was changed to datetime, incase any time depended insights were wanted.

***

## Exporting

Data from the cleaned `df_dog_clean` dataframe was exported to a .csv file named `twitter_archive_master.csv`